In [39]:
import torch
import numpy as np
from torch import nn

### Neural network for creating a instance of it
### And then loading the saved model state_dict()

In [40]:
class_names = ['0 - zero',
              '1 - one',
              '2 - two',
              '3 - three',
              '4 - four',
              '5 - five',
              '6 - six',
              '7 - seven',
              '8 - eight',
              '9 - nine']

In [41]:
class MNISTDigitRecognitionModel(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               output_shape:int):
    super().__init__()
    self.conv2d_block_1 = nn.Sequential(
       nn.Conv2d(in_channels=input_shape,
                 out_channels=hidden_units,
                 kernel_size=3,
                 stride=1,
                 padding=1),
       nn.ReLU(),
       nn.Conv2d(in_channels=hidden_units,
                 out_channels=hidden_units,
                 kernel_size=3,
                 stride=1,
                 padding=1),
       nn.ReLU(),
       nn.MaxPool2d(kernel_size=2,
                    stride=2)
    )
    self.conv2d_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                 out_channels=hidden_units,
                 kernel_size=3,
                 stride=1,
                 padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units * 7*7,
                  out_features=output_shape)
    )

  def forward(self, x):
    x = self.conv2d_block_1(x)
    # print(x.shape)
    x = self.conv2d_block_2(x)
    # print(x.shape)
    x = self.classifier(x)
    return x

In [42]:
torch.manual_seed(42)
model = MNISTDigitRecognitionModel(input_shape=1,
                                   hidden_units=10,
                                   output_shape=len(class_names))
model

MNISTDigitRecognitionModel(
  (conv2d_block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2d_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=490, out_features=10, bias=True)
  )
)

In [43]:
!pip install gradio

In [59]:
import gradio as gr
from PIL import Image
import torch
from torchvision.transforms import Compose, Grayscale, Resize, ToTensor
import os

# Load your PyTorch model and define class names
model_path = os.path.join('Model', 'DigitRecognitionCNNModel.pth')

class_names = ['0 - zero',
                '1 - one',
                '2 - two',
                '3 - three',
                '4 - four',
                '5 - five',
                '6 - six',
                '7 - seven',
                '8 - eight',
                '9 - nine']

# Create an instance of your model class
model = MNISTDigitRecognitionModel(input_shape=1,
                                   hidden_units=10,
                                   output_shape=len(class_names))

# Load the model state dictionary
checkpoint = "Model\DigitRecognitionCNNModel.pth"
model.load_state_dict(torch.load(checkpoint))

# Set the model to evaluation mode
model.eval()

# Define the transformation to preprocess input images
transform = Compose([
    Grayscale(num_output_channels=1),  # Convert to grayscale
    Resize((28, 28)),  # Resize to your model's input size
    ToTensor(),
])

# Define a function to preprocess the image and make predictions
def predict_digit(image):
    # Convert Gradio sketchpad image to PIL Image
    image = Image.fromarray((image * 255).astype('uint8'))
    image = transform(image)  # Preprocess the input image
    image = image.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        _, predicted = output.max(1)
    prediction = class_names[predicted.item()]
    return prediction

# Create a Gradio interface
iface = gr.Interface(fn=predict_digit, inputs="sketchpad", outputs="label")
iface.launch(debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
